# Nội suy ngược

<b>Bài toán:</b>
Tìm x biết $f(x)=y$ trong đó, hàm $f(x)$ chỉ có dạng bảng

<b>Ý tưởng:</b>

<ol>
<li>Xác định khoảng cách ly:
<ul>
    <li> Xác định các khoảng đơn điệu</li>
    <li> Xác định khoảng đơn điệu có chứa nghiệm</li>
</ul></li>
<li>Tìm nghiệm trên khoảng đơn điệu có chứa nghiệm hoặc khoảng <i>không</i> đơn điệu nhưng có nghiệm duy nhất</li>
</ol>

<b>Hai phương pháp tìm nghiệm</b>

PP1: Xây dựng xấp xỉ của hàm ngược \
    Hàm đơn điệu $[a,b] \rightarrow Im([a,b])$ là hàm song ánh => tồn tại ánh xạ ngược $Im([a,b]) \rightarrow [a,b]: x=g(y)$ \
    Điều kiện mốc nội suy $x_i \neq x_j \forall i \neq j \Rightarrow y_i \neq y_j \forall i \neq j$ \
    Xem y là ẩn, $x=g(y) \approx P_n(y)$ là hàm 

PP2: Phương pháp lặp để tìm nghiệm
<ul>
<li>Chọn 2 mốc liên tiếp chặn trên, chặn dưới của nghiệm</li>
<li>Nếu chọn mốc chặn dưới, xây dựng công thức lặp từ đa thức nội suy Newton tiến. Nếu chọn mốc chặn trên thì Newton lùi.(Chọn bên nào có ít mốc hơn)</li>
</ul>

Xét trường hợp chọn mốc bên trái, ta có:
$$P_n(x_k+th)=y_k+\Delta y_kt + \frac{\Delta^2y_k}{2!}t(t-1)+... = \bar y $$
Khi đó,
$$\begin{cases}
t &=& \frac{1}{\Delta y_k}[\bar y - y_k - (\frac{\Delta^2y_k}{2!}t(t-1)+...)] = \phi(t) \\
t_0 &=& \frac{\bar y - y_k}{\Delta y_k}, \hspace{.2cm} t_{i+1} = \phi(t_i)
\end{cases}$$
Sai số,
$$\frac{|t_{i+1}-t_i|}{|t_{i+1}|} < \delta$$

### Import packages

In [18]:
from termcolor        import colored
from math             import *
from sympy            import *
# from customDecorator  import * 

import matplotlib.pyplot as plt
import numpy             as np
import sys 
import time 

### Input chương trình

In [6]:
EPSILON_SYS   = 1e-15 
EPSILON       = 1e-6
MAXIMUM_POINT = 6 

In [4]:
def input_data():
    x_arr = []
    y_arr = []

    with open('input.txt', 'r+') as f:
        lines = f.readlines()
        for elm in lines[0].split(' '):
            x_arr.append(float(elm))
        for elm in lines[1].split(' '):
            y_arr.append(float(elm))
    
    return x_arr, y_arr

x, y = input_data()
print(x, y, sep='\n')

[1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1]
[0.171, 0.288, 0.357, 0.384, 0.375, 0.336, 0.273, 0.192, 0.099, 0.0, -0.099, -0.192, -0.273, -0.336, -0.375, -0.384, -0.357, -0.288, -0.171, 0.0, 0.231]


In [7]:
def Monnotonous_Parts(x_arr, y_arr):
    extreme_idx = [0]
    
    for i in range(1, len(y_arr) - 1):
        if y_arr[i] > max(y_arr[i - 1], y_arr[i + 1]) or y_arr[i] < min(y_arr[i - 1], y_arr[i + 1]):
            extreme_idx.append(i)
        else:
            continue
    
    extreme_idx.append(len(y_arr) - 1)

    mp_idx_list = [] 

    for j in range(len(extreme_idx) - 1):
        index_range = range(extreme_idx[j], extreme_idx[j + 1] + 1)
        mp_idx = [x for x in index_range] # mp is short for "monotonous partition"
        
        mp_idx_list.append(mp_idx) 

    return mp_idx_list

def Sub_Monotonous_Parts(x_arr, y_arr, value_y):
    mp_idx_list = Monnotonous_Parts(x_arr, y_arr)
    smp_idx_list = [] 
    
    for mp_idx in mp_idx_list:
        # Make sure the par is monotonically increase
        sign = 1 # sign = 1 if monotonically increase, sign = -1 if monotonically decrease
        if y_arr[mp_idx[0]] > y_arr[mp_idx[1]]:
            sign *= -1 

        if value_y > max(y_arr[mp_idx[0]], y_arr[mp_idx[-1]]) or value_y < min(y_arr[mp_idx[0]], y_arr[mp_idx[-1]]):
            continue 

        # Find the index of two neiboor nearest value of y using binary search
        left_idx  = mp_idx[0]
        right_idx = mp_idx[-1] 
            
        while True:
            idx = int((left_idx + right_idx) / 2) 
            if (value_y - y_arr[idx]) * sign > 0:
                left_idx = idx 
            else: 
                right_idx = idx 
                    
            if right_idx - left_idx == 1: 
                break 

        # return sub table of y 
        if left_idx + right_idx > mp_idx[0] + mp_idx[-1]:
            right_bound = right_idx 
            left_bound  = max(right_idx - MAXIMUM_POINT + 1, mp_idx[0]) 
            idx_range   = range(left_bound, right_bound + 1)
        
            smp = [i for i in idx_range][::-1] # Short for "sub monotonous partition"
        else:
            left_bound  = left_idx 
            right_bound = min(left_idx + MAXIMUM_POINT - 1, mp_idx[-1])
            idx_range   = range(left_bound, right_bound + 1)
        
            smp = [i for i in idx_range]

        smp_idx_list.append(smp)  

    return smp_idx_list

test = Sub_Monotonous_Parts(x, y, 0.2)
print(test)

[[0, 1, 2, 3], [6, 7, 8, 9, 10, 11], [20, 19, 18, 17, 16, 15]]
